In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile
import glob
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, BatchNormalization, concatenate, Input
from tensorflow.keras import Model
from tensorflow import keras

import warnings
warnings.filterwarnings("ignore")


In [3]:
path=  '/content/drive/My Drive/Colab Notebooks/train.zip'

In [4]:
zip_file = zipfile.ZipFile(path)
zip_file.extractall('.')



In [5]:
zip_file = zipfile.ZipFile('/content/drive/My Drive/Colab Notebooks/test.zip')
zip_file.extractall('./test/')

In [6]:
train_files = glob.glob('/content/train/*.npy')
len(train_files)

62735

In [15]:
def trainGenerator():
    for file in tqdm(train_files):
        dataset = np.load(file)
        target= dataset[:,:,-1].reshape(120,120,1)
        remove_minus = np.where(target < 0, 0, target)
        feature = dataset[:,:,:4]

        yield (feature, remove_minus)
        
train_dataset = tf.data.Dataset.from_generator(trainGenerator, (tf.float32, tf.float32), (tf.TensorShape([120,120,4]),tf.TensorShape([120,120,1])))
train_dataset = train_dataset.batch(100).prefetch(1)

In [16]:
train_dataset

<PrefetchDataset shapes: ((None, 120, 120, 4), (None, 120, 120, 1)), types: (tf.float32, tf.float32)>

In [9]:
test_path = '/content/test'
test_files = sorted(glob.glob(test_path + '/*.npy'))

X_test = []

for file in tqdm(test_files, desc = 'test'):
    data = np.load(file)
    X_test.append(data)

X_test = np.array(X_test)

test: 100%|██████████| 2674/2674 [00:00<00:00, 2926.24it/s]


In [21]:
def base_model(input_layer, start_neurons):

    # conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(input_layer)
    # pool1 = BatchNormalization()(conv1)
    # pool1 = keras.layers.Dropout(0.8)(pool1)
    # pool1 = MaxPooling2D((2, 2))(pool1)

    # conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
    # pool2 = BatchNormalization()(conv2)
    # pool2 = keras.layers.Dropout(0.8)(pool2)
    # pool2 = MaxPooling2D((2, 2))(pool2)

    # convm = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(pool2)

    # deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(convm)
    # uconv2 = concatenate([deconv2, conv2])
    # uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    # uconv2 = BatchNormalization()(uconv2)

    # deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    # uconv1 = concatenate([deconv1, conv1])
    # uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    # uconv1 = BatchNormalization()(uconv1)
    # output_layer = Conv2D(1, (1,1), padding="same", activation='relu')(uconv1)
    
    # return output_layer
    
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(input_layer)
    pool1 = BatchNormalization()(conv1)
    pool1 = keras.layers.Dropout(0.8)(pool1)
    pool1 = MaxPooling2D((2, 2))(pool1)
    


    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
    pool2 = BatchNormalization()(conv2)
    pool2 = keras.layers.Dropout(0.6)(pool2)
    pool2 = MaxPooling2D((2, 2))(pool2)
    


    conv3 = Conv2D(start_neurons * 3, (3, 3), activation="relu", padding="same")(pool2)
    pool3 = BatchNormalization()(conv3)
    pool3 = keras.layers.Dropout(0.6)(pool3)
    pool3 = MaxPooling2D((2, 2))(pool2)
    



    convm = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(pool3)

    deconv3 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv3 = concatenate([deconv3, conv3])
    uconv3 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv3)
    uconv3 = BatchNormalization()(uconv3)



    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv3)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = BatchNormalization()(uconv2)

    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = BatchNormalization()(uconv1)


    


    output_layer = Conv2D(1, (1,1), padding="same", activation='relu')(uconv1)
    
    return output_layer

input_layer = Input((120, 120, 4))
output_layer = base_model(input_layer,64)

In [22]:
model = Model(input_layer, output_layer)
model.compile(loss='mae', optimizer='adam')

In [23]:
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 120, 120, 4) 0                                            
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 120, 120, 64) 2368        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_8 (BatchNor (None, 120, 120, 64) 256         conv2d_12[0][0]                  
__________________________________________________________________________________________________
dropout_4 (Dropout)             (None, 120, 120, 64) 0           batch_normalization_8[0][0]      
_______________________________________________________________________________________

In [24]:
my_model=model.fit(train_dataset, epochs = 5, verbose=1)

Epoch 1/5





  0%|          | 0/62735 [00:00<?, ?it/s]


  0%|          | 100/62735 [00:00<01:07, 930.69it/s]


  0%|          | 191/62735 [00:00<01:07, 922.50it/s]

      1/Unknown - 0s 73ms/step - loss: 12.1356




  0%|          | 238/62735 [00:01<07:33, 137.93it/s]

      2/Unknown - 1s 301ms/step - loss: 12.6493WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.2095s vs `on_train_batch_end` time: 0.3187s). Check your callbacks.


...

625/628 [============================>.] - ETA: 1s - loss: 3.1485




100%|█████████▉| 62600/62735 [05:31<00:00, 136.62it/s]


100%|█████████▉| 62693/62735 [05:31<00:00, 183.55it/s]

626/628 [============================>.] - ETA: 1s - loss: 3.1484

100%|██████████| 62735/62735 [05:32<00:00, 188.85it/s]


628/628 [==============================] - 333s 530ms/step - loss: 3.1481


In [25]:
test_path = '/content/test'
test_files = sorted(glob.glob(test_path + '/*.npy'))

X_test = []

for file in tqdm(test_files, desc = 'test'):
    data = np.load(file)
    X_test.append(data)

X_test = np.array(X_test)




test:   0%|          | 0/2674 [00:00<?, ?it/s]


test:   3%|▎         | 86/2674 [00:00<00:03, 858.47it/s]


test:   6%|▌         | 150/2674 [00:00<00:03, 777.37it/s]


test:   8%|▊         | 201/2674 [00:00<00:03, 671.69it/s]


test:  10%|▉         | 267/2674 [00:00<00:03, 667.32it/s]


test:  12%|█▏        | 327/2674 [00:00<00:03, 645.23it/s]


test:  14%|█▍        | 386/2674 [00:00<00:03, 627.58it/s]


test:  17%|█▋        | 446/2674 [00:00<00:03, 618.22it/s]


test:  19%|█▉        | 507/2674 [00:00<00:03, 614.99it/s]


test:  21%|██        | 565/2674 [00:00<00:03, 597.04it/s]


test:  23%|██▎       | 625/2674 [00:01<00:03, 597.72it/s]


test:  26%|██▌       | 691/2674 [00:01<00:03, 614.57it/s]


test:  28%|██▊       | 752/2674 [00:01<00:03, 596.34it/s]


test:  31%|███       | 823/2674 [00:01<00:02, 625.68it/s]


test:  33%|███▎      | 886/2674 [00:01<00:03, 516.34it/s]


test:  36%|███▌      | 950/2674 [00:01<00:03, 547.47it/s]


test:  38%|███▊      | 1008/2674 [00:01<00:03, 55

In [26]:
X_test.shape

(2674, 120, 120, 4)

In [33]:
pred = model.predict(X_test)

In [42]:
submission2 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/sample_submission.csv')

In [43]:
submission2.iloc[:,1:] = pred.reshape(-1, 14400).astype(int)
submission2.to_csv('/content/drive/My Drive/Colab Notebooks/Dacon_baselinehello.csv', index = False)